<a href="https://colab.research.google.com/github/KevinNyiringango/Initial_repo/blob/main/chatbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
import pandas as pd
from sklearn.model_selection import train_test_split
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
import re
import matplotlib.pyplot as plt
import seaborn as sns
import json
import random
from transformers import AutoTokenizer
from datasets import Dataset

In [8]:
import json
import random
from transformers import AutoTokenizer
from datasets import Dataset

# Load the dataset
with open("Bigdata.jsonl", "r") as file:
    data = [json.loads(line) for line in file]

# Prepare conversation pairs
conversations = []
for entry in data:
    user_input = entry["prompt"]
    bot_response = entry["completion"]
    conversations.append(f"User: {user_input}\nBot: {bot_response}")

# Load GPT-2 tokenizer
model_name = "gpt2"
tokenizer = AutoTokenizer.from_pretrained(model_name)

# ✅ Fix: Set padding token explicitly
tokenizer.pad_token = tokenizer.eos_token

# Tokenize conversation data
tokenized_data = tokenizer(conversations, padding=True, truncation=True, return_tensors="pt")

# Convert tokenized data into a Hugging Face Dataset
dataset = Dataset.from_dict({
    "input_ids": tokenized_data["input_ids"],
    "attention_mask": tokenized_data["attention_mask"]
})

# Print a sample to verify everything works
print(dataset[0])

# Now you can train your model with this dataset


{'input_ids': [12982, 25, 1374, 466, 345, 2190, 9875, 14129, 30, 198, 20630, 25, 632, 318, 7151, 284, 3424, 262, 11699, 351, 47375, 290, 788, 11662, 5849, 72, 34, 495, 11662, 319, 262, 5676, 1989, 5403, 257, 1110, 329, 1936, 1528, 13, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50

In [9]:
import os
os.environ["WANDB_DISABLED"] = "true"

In [10]:
import torch
from transformers import AutoModelForCausalLM, Trainer, TrainingArguments, DataCollatorForLanguageModeling
from datasets import Dataset

model = AutoModelForCausalLM.from_pretrained("gpt2")

tokenized_data = tokenizer(conversations, padding=True, truncation=True, return_tensors="pt")

dataset = Dataset.from_dict({
    "input_ids": tokenized_data["input_ids"],
    "attention_mask": tokenized_data["attention_mask"]
})

training_args = TrainingArguments(
    output_dir="./chatbot_model",
    per_device_train_batch_size=2,
    num_train_epochs=1,
    save_steps=500,
    save_total_limit=2,
    logging_dir="./logs",
    report_to="none"
)

data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=False
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset,
    data_collator=data_collator,
)

trainer.train()


model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

`loss_type=None` was set in the config but it is unrecognised.Using the default loss: `ForCausalLMLoss`.


Step,Training Loss
500,2.125700
1000,1.801900


Step,Training Loss
500,2.125700
1000,1.801900
1500,1.659300
2000,1.634600
2500,1.639500


TrainOutput(global_step=2500, training_loss=1.7721912841796874, metrics={'train_runtime': 816.4786, 'train_samples_per_second': 6.124, 'train_steps_per_second': 3.062, 'total_flos': 1076808960000000.0, 'train_loss': 1.7721912841796874, 'epoch': 1.0})

In [11]:
model.save_pretrained("chatbot_model")
tokenizer.save_pretrained("chatbot_model")

('chatbot_model/tokenizer_config.json',
 'chatbot_model/special_tokens_map.json',
 'chatbot_model/vocab.json',
 'chatbot_model/merges.txt',
 'chatbot_model/added_tokens.json',
 'chatbot_model/tokenizer.json')

In [ ]:
# Ensure model is on the correct device (GPU if available)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

def chatbot_response(user_input):
    input_text = f"User: {user_input}\nBot:"
    inputs = tokenizer(input_text, return_tensors="pt", padding=True, truncation=True)

    # Move tensors to the same device as the model
    input_ids = inputs.input_ids.to(device)
    attention_mask = inputs.attention_mask.to(device)

    # Generate response
    output = model.generate(
        input_ids,
        attention_mask=attention_mask,
        max_length=50,
        pad_token_id=tokenizer.eos_token_id
    )

    response = tokenizer.decode(output[:, input_ids.shape[-1]:][0], skip_special_tokens=True)
    return response

# Test chatbot
while True:
    user_input = input("You: ")
    if user_input.lower() in ["quit", "exit"]:
        break
    print("Bot:", chatbot_response(user_input))


In [14]:
from google.colab import drive
drive.mount('/content/drive')
!cp -r chatbot_model /content/drive/MyDrive/chatbot

Mounted at /content/drive
